In [1]:
 !nvidia-smi

Tue Jul  9 12:30:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [22]:
%%writefile debubble.cu
#include <cub/cub.cuh>


void SORT(int* nums, size_t n, int* output) {
    void* d_temp_storage = nullptr;
    size_t temp_storage_bytes = 0;
    cub::DeviceRadixSort::SortKeys(d_temp_storage, temp_storage_bytes, nums, output, n);
    cudaMalloc(&d_temp_storage, temp_storage_bytes);
    cub::DeviceRadixSort::SortKeys(d_temp_storage, temp_storage_bytes, nums, output, n);
    cudaFree(d_temp_storage);
}
int findleft(int* a, int l, int r){
  if(l == r){
    return l;
  }
  int mid = (l + r) / 2;
  if(a[mid] < 0){
    return findleft(a, mid + 1, r);
  }
  return findleft(a, l, mid);
}
int findright(int* a, int l, int r){
  if(l == r){
    return l;
  }
  int mid = (l + r) / 2;
  if(a[mid] <= 0){
    return findright(a, mid + 1, r);
  }
  return findright(a, l, mid);
}
int main() {
    int num_elements = 5;
    int* keys = new int[num_elements];
    int* sorted_keys = new int[num_elements];
    for (int i = 0; i < num_elements; ++i) {
        keys[i] = num_elements - i;
    }
    int* d_keys;
    int* d_sorted_keys;
    cudaMalloc(&d_keys, num_elements * sizeof(int));
    cudaMalloc(&d_sorted_keys, num_elements * sizeof(int));
    cudaMemcpy(d_keys, keys, num_elements * sizeof(int), cudaMemcpyHostToDevice);
    SORT(d_keys, num_elements, d_sorted_keys);
    cudaMemcpy(sorted_keys, d_sorted_keys, num_elements * sizeof(int), cudaMemcpyDeviceToHost);
    for (int i = 0; i < num_elements; i++) {
      std::cout << sorted_keys[i] << " ";
    }
    int left = findleft(sorted_keys, 0 ,num_elements - 1);
    int right = findright(sorted_keys, 0 ,num_elements - 1);
    int* debubbled_keys = new int[num_elements];
    std::cout<<left<<" "<<right<<std::endl;
    int len = 0;
    if(sorted_keys[0] <= 0 && sorted_keys[num_elements - 1] >= 0){
      for(int i = 0;i < left; i++){
        debubbled_keys[i] = sorted_keys[i];
        len++;
      }
      for(int i = 0; i< num_elements - right; i++){
        debubbled_keys[i + left] = sorted_keys[i + right];
        len++;
      }
    }
    else{
      for(int i = 0;i < num_elements; i++){
        debubbled_keys[i] = sorted_keys[i];
        len++;
      }
    }
    for(int i = 0; i < len; i++){
      std::cout<<debubbled_keys[i]<<std::endl;
    }
    delete[] keys;
    delete[] sorted_keys;
    delete[] debubbled_keys;
    return 0;
}





Overwriting debubble.cu


In [25]:
!nvcc -std=c++14 -arch=sm_75 debubble.cu -o debubble

In [26]:
!time ./debubble

1 2 3 4 5 0 0
1
2
3
4
5

real	0m0.132s
user	0m0.008s
sys	0m0.117s
